In [1]:
import io

import netaddr

import pandas as pd

import aiohttp

from rpki_analysis.delegated_stats import read_delegated_stats, read_delegated_extended_stats

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.afrinic.net/pub/stats/afrinic/2024/delegated-afrinic-20240214') as resp:
        df_delegated = read_delegated_stats(io.StringIO(await resp.text()))

async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.afrinic.net/pub/stats/afrinic/2024/delegated-afrinic-extended-20240214') as resp:
        df_extended = read_delegated_extended_stats(io.StringIO(await resp.text()))



In [3]:
extended_keys = set(df_extended.keys())
delegated_keys = set(df_delegated.keys())
# resource is often not printable, remove it
overlap = (extended_keys & delegated_keys) - set(['resource', 'country'])

lhs = df_extended[overlap]
lhs = lhs[(lhs.status != 'available') & (lhs.status != 'reserved')]
rhs = df_delegated[overlap]
rhs = rhs[(rhs.status != 'available') & (rhs.status != 'reserved')]

lhs_set = set(lhs.itertuples(index=False))
rhs_set = set(rhs.itertuples(index=False))

/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_99041/2954961098.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  lhs = df_extended[overlap]
/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_99041/2954961098.py:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  rhs = df_delegated[overlap]


## Difference

In [4]:
display(lhs_set - rhs_set)
display(rhs_set - lhs_set)

set()

set()

# Now with country

In [5]:
extended_keys = set(df_extended.keys())
delegated_keys = set(df_delegated.keys())
# resource is often not printable, remove it
overlap = (extended_keys & delegated_keys) - set(['resource'])

lhs = df_extended[overlap]
lhs = lhs[(lhs.status != 'available') & (lhs.status != 'reserved')]
rhs = df_delegated[overlap]
rhs = rhs[(rhs.status != 'available') & (rhs.status != 'reserved')]

lhs_set = set(lhs.itertuples(index=False))
rhs_set = set(rhs.itertuples(index=False))

/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_99041/3043363742.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  lhs = df_extended[overlap]
/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_99041/3043363742.py:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  rhs = df_delegated[overlap]


## Difference

In [6]:
display(len(lhs_set - rhs_set))
display(len(rhs_set - lhs_set))

42

42

In [7]:
pd.DataFrame.from_records(list(lhs_set - rhs_set)).to_csv('extended_not_delegated.csv')
pd.DataFrame.from_records(list(rhs_set - lhs_set)).to_csv('delegated_not_extended.csv')